In [1]:
from telnetlib import X3PAD
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pickle 
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import pickle 
from sklearn.metrics import mean_squared_error

In [2]:
df_2020 = pd.read_csv('merged_gw2020.csv',low_memory = False)
df_2021 = pd.read_csv('merged_gw2021.csv',low_memory = False)




In [3]:
season_2020 = pd.DataFrame({'season' : ['2019/20']*df_2020.shape[0]   })
season_2021 = pd.DataFrame({'season' : ['2020/21']*df_2021.shape[0]   })


In [4]:
df_2020 =pd.concat([df_2020, season_2020] ,axis = 'columns')
df_2021 =pd.concat([df_2021, season_2021] ,axis = 'columns')

In [5]:
df= pd.concat([df_2020 , df_2021], axis = 'index',ignore_index = True)





In [6]:
df["was_home"] = df["was_home"].astype(int)


In [7]:
df.sort_values(by=['name','season','GW'], inplace =True)
df.reset_index(inplace = True)


df.drop(['index'],axis= 'columns' , inplace =True)



In [8]:
df['id'] = df.groupby(['name']).ngroup()



In [9]:
df['pos']= df.groupby(['position']).ngroup()



In [10]:
df['teams'] = df.groupby(['team']).ngroup()



In [11]:
identity = df['id'].unique()




enter value of x where x is the latest gameweek

In [12]:
x = 35

In [13]:
filt = (df['GW'] == x) & (df['season']=='2020/21')
df_points = df.loc[filt , ['total_points','id' ]]
df_points.drop_duplicates(subset='id',keep = 'first',inplace = True)
df_points.reset_index(inplace = True)


df_points.drop(['index'],axis= 'columns' , inplace =True)
df_points


,total_points,id
0,0,0
1,0,1
2,2,2
3,2,4
4,0,5
...,...,...
723,0,975
724,0,976
725,0,978
726,0,980


In [14]:
y=x-1
Gameweeks=[]


In [15]:
for i in range(5):
    Gameweeks.append(y)
    y=y-1




In [16]:
dataframes = {}

In [17]:
for GW in Gameweeks:
    filt=(df['GW'] == GW) & (df['season'] == '2020/21')
    dataframes[GW] = df[filt]
    dataframes[GW].reset_index(inplace = True)
    dataframes[GW].drop(['index'],axis= 'columns' , inplace =True)
    dataframes[GW].drop([ 'kickoff_time','season', 'name' , 'position','pos' ,'GW' ,'team','teams','round','element'],axis= 'columns' , inplace =True)

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
dataframes[34]

,xP,assists,bonus,bps,clean_sheets,creativity,fixture,goals_conceded,goals_scored,ict_index,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,id
0,0.0,0,0,0,0,0.0,333,0,0,0.0,...,2,0.0,0,-48,0,48,53,1,0,0
1,3.0,0,0,15,0,1.8,335,1,0,1.5,...,1,0.0,2,-27000,9050,36050,55,0,0,1
2,0.8,0,0,3,0,10.8,334,0,0,5.7,...,1,38.0,1,-1827,111,1938,48,1,0,2
3,1.6,0,0,17,0,0.0,331,1,0,2.4,...,3,0.0,3,-70200,11288,81488,51,1,0,4
4,0.0,0,0,0,0,0.0,337,0,0,0.0,...,0,0.0,0,-98,0,98,44,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,0.0,0,0,0,0,0.0,331,0,0,0.0,...,3,0.0,0,49,104,55,40,1,0,978
790,2.8,0,0,0,0,0.0,331,0,0,0.0,...,3,0.0,0,0,0,0,40,0,0,980
791,2.8,0,0,0,0,0.0,364,0,0,0.0,...,1,0.0,0,0,0,0,40,1,0,980
792,-1.0,0,0,0,0,0.0,332,0,0,0.0,...,0,0.0,0,-1150,114,1264,45,1,0,980


In [19]:
n = 0
for keys in dataframes:
    column = []
    
    for value in dataframes[keys].columns:
        
        if (value != 'id')  & (value!='team'):
            value = value +f'_{n}' 
        column.append(value)
        
    dataframes[keys].columns = column
    n=n-1       



In [20]:
for keys in dataframes:
    dataframes[keys].drop_duplicates(subset='id' , keep ='first' , inplace = True)
    dataframes[keys].reset_index(inplace = True)
    dataframes[keys].drop(['index'],axis= 'columns' , inplace =True)



C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [21]:
df1 = pd.merge(dataframes[x-1] , dataframes[x-2], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-3], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-4], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-5], on = 'id' , how = 'outer')

In [22]:
filt = (df['GW'] == x-1) | (df['GW'] == x-2) | (df['GW'] == x-3) | (df['GW'] == x-4) | (df['GW'] == x-5)
df_position = df.loc[filt , ['id' , 'pos','teams']]



In [23]:
df_position.drop_duplicates(subset='id',keep = 'first',inplace = True)
df_position.reset_index(inplace = True)
df_position.drop(['index'],axis= 'columns' , inplace =True)

In [24]:
df1.drop_duplicates(subset='id',keep = 'first',inplace = True)
df1.reset_index(inplace = True)
df1.drop(['index'],axis= 'columns' , inplace =True)



In [25]:
df1 = pd.merge(df1 , df_position, on = 'id' , how = 'inner')



In [26]:
filt = df['GW']== x 
df_points = df.loc[filt,['id' , 'total_points']]
df_points.drop_duplicates(subset='id',keep = 'first',inplace = True)
df_points.reset_index(inplace = True)
df_points.drop(['index'],axis= 'columns' , inplace =True)



In [27]:
df1 = pd.merge(df1 , df_points, on = 'id' , how = 'inner')




In [28]:

df1 = df1.fillna(df1.median())



In [29]:
columns = df1.columns
remove = ('pos','teams','id','total_points')
a = set(columns)
b = a.difference(remove)
columns = list(b)

    

In [30]:
df1

,xP_0,assists_0,bonus_0,bps_0,clean_sheets_0,creativity_0,fixture_0,goals_conceded_0,goals_scored_0,ict_index_0,...,total_points_-4,transfers_balance_-4,transfers_in_-4,transfers_out_-4,value_-4,was_home_-4,yellow_cards_-4,pos,teams,total_points
0,0.0,0,0,0,0,0.0,333,0,0,0.0,...,0.0,11.5,229.5,351.5,45.0,0.0,0.0,1,3,0
1,3.0,0,0,15,0,1.8,335,1,0,1.5,...,1.0,18750.0,35570.0,16820.0,54.0,0.0,0.0,0,21,1
2,0.8,0,0,3,0,10.8,334,0,0,5.7,...,0.0,11.5,229.5,351.5,45.0,0.0,0.0,3,4,2
3,1.6,0,0,17,0,0.0,331,1,0,2.4,...,0.0,43844.0,61455.0,17611.0,51.0,0.0,0.0,2,16,3
4,0.0,0,0,0,0,0.0,337,0,0,0.0,...,0.0,-139.0,0.0,139.0,44.0,1.0,0.0,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,0.0,0,0,0,0,0.0,331,0,0,0.0,...,0.0,42.0,146.0,104.0,40.0,0.0,0.0,0,0,0
722,1.5,0,0,0,0,0.0,339,0,0,0.0,...,0.0,11.5,229.5,351.5,45.0,0.0,0.0,2,12,0
723,0.0,0,0,0,0,0.0,331,0,0,0.0,...,0.0,54.0,103.0,49.0,40.0,0.0,0.0,0,0,0
724,2.8,0,0,0,0,0.0,331,0,0,0.0,...,0.0,-1300.0,661.0,1961.0,45.0,0.0,0.0,2,2,0


In [31]:
df1.loc[:,columns] = StandardScaler().fit_transform(df1.loc[:, columns])



In [32]:
X=df1.drop(['id' ,'total_points' ,'pos','teams'],axis ='columns') 
Y= df1.loc[:, {'total_points'}]






In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2 )

In [34]:
X

,xP_0,assists_0,bonus_0,bps_0,clean_sheets_0,creativity_0,fixture_0,goals_conceded_0,goals_scored_0,ict_index_0,...,team_a_score_-4,team_h_score_-4,threat_-4,total_points_-4,transfers_balance_-4,transfers_in_-4,transfers_out_-4,value_-4,was_home_-4,yellow_cards_-4
0,-0.574426,-0.146286,-0.201934,-0.510777,-0.422577,-0.417488,-0.870225,-0.359250,-0.124669,-0.497684,...,-0.336893,0.184183,-0.224389,-0.264138,-0.142241,-0.205170,-0.121384,-0.246170,-0.497416,-0.150117
1,0.826424,-0.146286,-0.201934,1.022399,-0.422577,-0.225911,-0.185736,0.769821,-0.124669,0.034188,...,-0.336893,1.562705,0.321361,0.368747,0.390787,0.835279,0.922611,1.022342,-0.497416,-0.150117
2,-0.200866,-0.146286,-0.201934,-0.204142,-0.422577,0.731974,-0.527980,-0.359250,-0.124669,1.523430,...,-0.336893,0.184183,-0.224389,-0.264138,-0.142241,-0.205170,-0.121384,-0.246170,-0.497416,-0.150117
3,0.172694,-0.146286,-0.201934,1.226823,-0.422577,-0.417488,-1.554714,0.769821,-0.124669,0.353312,...,-0.336893,-2.572863,-0.224389,-0.264138,1.104602,1.597351,0.972755,0.599505,-0.497416,-0.150117
4,-0.574426,-0.146286,-0.201934,-0.510777,-0.422577,-0.417488,0.498753,-0.359250,-0.124669,-0.497684,...,-0.336893,-2.572863,-0.224389,-0.264138,-0.146522,-0.211926,-0.134855,-0.387116,2.010390,-0.150117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,-0.574426,-0.146286,-0.201934,-0.510777,-0.422577,-0.417488,-1.554714,-0.359250,-0.124669,-0.497684,...,-0.336893,-2.572863,-0.224389,-0.264138,-0.141373,-0.207628,-0.137074,-0.950899,-0.497416,-0.150117
722,0.125999,-0.146286,-0.201934,-0.510777,-0.422577,-0.417488,1.183242,-0.359250,-0.124669,-0.497684,...,-0.336893,0.184183,-0.224389,-0.264138,-0.142241,-0.205170,-0.121384,-0.246170,-0.497416,-0.150117
723,-0.574426,-0.146286,-0.201934,-0.510777,-0.422577,-0.417488,-1.554714,-0.359250,-0.124669,-0.497684,...,-0.336893,-2.572863,-0.224389,-0.264138,-0.141032,-0.208894,-0.140561,-0.950899,-0.497416,-0.150117
724,0.733034,-0.146286,-0.201934,-0.510777,-0.422577,-0.417488,-1.554714,-0.359250,-0.124669,-0.497684,...,-0.336893,0.184183,-0.224389,-0.264138,-0.179547,-0.192466,-0.019352,-0.246170,-0.497416,-0.150117


In [34]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_depth=20, random_state=0)

model.fit(X_train ,Y_train)



<ipython-input-34-c85b9c503331>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train ,Y_train)


RandomForestRegressor(max_depth=20, random_state=0)

In [35]:
prediction = model.predict(X_test)



In [36]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test,prediction)



4.9310789804002315

In [37]:
pred_training = model.predict(X_train)
mean_squared_error(Y_train,pred_training)



0.5788689276769307

In [38]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(Y_test,prediction)



1.1509210708996667

In [39]:
with open('model_pickle','wb') as file:
    pickle.dump(model,file)

